<a href="https://colab.research.google.com/github/Alexandre-Data/OC-Projet-7/blob/main/Projet7_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from google.colab import drive
drive.mount('/content/drive')
!git version
!pip install pyngrok
!pip install mlflow
!pip install mlflow-colab
!pip install SMOTE
!pip install -U imbalanced-learn
import numpy as np
import pandas as pd
import gc
import time
from contextlib import contextmanager
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import confusion_matrix, make_scorer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

from pyngrok import ngrok
warnings.simplefilter(action='ignore', category=FutureWarning)
from sklearn.model_selection import GridSearchCV, KFold, StratifiedKFold

from sklearn.pipeline import Pipeline
from imblearn.pipeline import make_pipeline
from imblearn.over_sampling import SMOTE
import smote
import mlflow

from sklearn.linear_model import LogisticRegression


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
git version 2.25.1
ERROR: Could not find a version that satisfies the requirement mlflow_colab (from versions: none)
ERROR: No matching distribution found for mlflow_colab


In [10]:
!pip install mlflow
import mlflow
!pip install mlflow-colab

ERROR: Could not find a version that satisfies the requirement mlflow-colab (from versions: none)
ERROR: No matching distribution found for mlflow-colab


In [7]:
from mlflow_colab import *
mlf_colab.init("/content/drive/MyDrive/OPENCLASSROOM/PROJET 7/")

ModuleNotFoundError: ignored

In [1]:
df=pd.read_csv("/content/drive/MyDrive/OPENCLASSROOM/PROJET 7/top_25_features.csv")

NameError: ignored

In [3]:
df.head()

,index,SK_ID_CURR,TARGET,EXT_SOURCE_2,EXT_SOURCE_3,index.1,DAYS_BIRTH,DAYS_REGISTRATION,SK_ID_CURR.1,DAYS_ID_PUBLISH,...,POS_MONTHS_BALANCE_MEAN,BURO_DAYS_CREDIT_MIN,BURO_DAYS_CREDIT_UPDATE_MEAN,EXT_SOURCE_1,PREV_HOUR_APPR_PROCESS_START_MEAN,ACTIVE_DAYS_CREDIT_ENDDATE_MAX,BURO_DAYS_CREDIT_ENDDATE_MIN,AMT_INCOME_TOTAL,PREV_AMT_CREDIT_MEAN,APPROVED_AMT_ANNUITY_MAX
0,0,100002,1.0,0.262949,0.139376,0,-9461,-3648.0,100002,-2120,...,-10.000000,-1437.0,-499.875,0.083037,9.000000,780.0,-1072.0,202500.0,179055.00,9251.775
1,1,100003,0.0,0.622246,0.000000,1,-16765,-1186.0,100003,-291,...,-43.785714,-2586.0,-816.000,0.311267,14.666667,1216.0,-2434.0,270000.0,484191.00,98356.995
2,2,100004,0.0,0.555912,0.729567,2,-19046,-4260.0,100004,-2531,...,-25.500000,-1326.0,-532.000,0.000000,5.000000,0.0,-595.0,67500.0,20106.00,5357.250
3,3,100006,0.0,0.650442,0.000000,3,-19005,-9833.0,100006,-2437,...,-9.619048,0.0,0.000,0.000000,14.666667,0.0,0.0,135000.0,291695.50,39954.510
4,4,100007,0.0,0.322738,0.000000,4,-19932,-4311.0,100007,-3458,...,-33.636364,-1149.0,-783.000,0.000000,12.333333,0.0,-783.0,121500.0,166638.75,22678.785


In [4]:
non_finite_values = np.isfinite(df)
df = df.replace([np.inf, -np.inf], np.nan).dropna()


In [5]:
#CODE SANS SMOTE
X = df.drop('TARGET', axis=1)
y = df['TARGET']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
dt = DecisionTreeClassifier(random_state=42)

# On fit
dt.fit(X_train, y_train)

# On prédit
y_pred = dt.predict(X_test)

In [6]:
# Calculer la matrice de confusion
confusion_mat = confusion_matrix(y_test, y_pred)

# Afficher la matrice de confusion
print("Matrice de confusion :")
print(confusion_mat)

Matrice de confusion :
[[61251  5034]
 [ 4192   774]]


In [7]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Calculer différents scores
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

# Afficher les scores
print(f'Accuracy: {accuracy*100:.2f}%')
print(f'Precision: {precision*100:.2f}%')
print(f'Recall: {recall*100:.2f}%')
print(f'F1-Score: {f1*100:.2f}%')
print(f'ROC AUC Score: {roc_auc*100:.2f}%')


Accuracy: 87.05%
Precision: 13.33%
Recall: 15.59%
F1-Score: 14.37%
ROC AUC Score: 54.00%


In [8]:
#CODE AVEC SMOTE
X = df.drop('TARGET', axis=1)
y = df['TARGET']

# split train test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# SMOTE sur l'ensemble d'entraînement
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Créez une instance de DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)

# Ajustez le modèle à l'ensemble d'entraînement suréchantillonné
dt.fit(X_train_resampled, y_train_resampled)

# Faites des prédictions sur l'ensemble de test
y_pred = dt.predict(X_test)

In [9]:
# Calculer la matrice de confusion
confusion_mat = confusion_matrix(y_test, y_pred)

# Afficher la matrice de confusion
print("Matrice de confusion :")
print(confusion_mat)

Matrice de confusion :
[[55144 11141]
 [ 3439  1527]]


In [10]:
# Calculer différents scores
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

# Afficher les scores
print(f'Accuracy: {accuracy*100:.2f}%')
print(f'Precision: {precision*100:.2f}%')
print(f'Recall: {recall*100:.2f}%')
print(f'F1-Score: {f1*100:.2f}%')
print(f'ROC AUC Score: {roc_auc*100:.2f}%')


Accuracy: 79.54%
Precision: 12.05%
Recall: 30.75%
F1-Score: 17.32%
ROC AUC Score: 56.97%


In [13]:
X = df.drop('TARGET', axis=1)
y = df['TARGET']

# Divisez les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Appliquer SMOTE uniquement sur l'ensemble d'entraînement
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Définir les hyperparamètres à rechercher
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Créer une instance de DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)

# Effectuer la recherche par grille avec validation croisée
with mlflow.start_run():
    grid_search = GridSearchCV(dt, param_grid, scoring='roc_auc', cv=2)
    grid_search.fit(X_train_resampled, y_train_resampled)

    # Obtenir les meilleurs hyperparamètres et le meilleur modèle
    best_params = grid_search.best_params_
    best_model = grid_search.best_estimator_

    # Faire des prédictions sur l'ensemble de test avec le meilleur modèle
    y_pred = best_model.predict(X_test)

    # Évaluer les performances du meilleur modèle
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred)

    # Enregistrer les hyperparamètres et les scores dans MLflow
    mlflow.log_params(best_params)
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1", f1)
    mlflow.log_metric("roc_auc", roc_auc)


In [14]:
roc_auc

0.6055738044120553

In [15]:
best_model

DecisionTreeClassifier(criterion='entropy', max_depth=15, min_samples_leaf=4,
                       min_samples_split=10, random_state=42)

In [1]:
X = df.drop('TARGET', axis=1)
y = df['TARGET']

# Divisez les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Appliquer SMOTE uniquement sur l'ensemble d'entraînement
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Définir les hyperparamètres à rechercher
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

def custom_loss(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    fn_weight = 10.0  # poids pour les faux négatifs
    fp_weight = 10.0  # poids pour les faux positifs
    return cm[0, 1] * fp_weight + cm[1, 0] * fn_weight

custom_scorer = make_scorer(custom_loss, greater_is_better=False)

# Créer une instance de DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)

# Effectuer la recherche par grille avec validation croisée
with mlflow.start_run():
    grid_search = GridSearchCV(dt, param_grid, scoring=custom_scorer, cv=2)
    grid_search.fit(X_train_resampled, y_train_resampled)

    # Obtenir les meilleurs hyperparamètres et le meilleur modèle
    best_params = grid_search.best_params_
    best_model = grid_search.best_estimator_

    # Faire des prédictions sur l'ensemble de test avec le meilleur modèle
    y_pred = best_model.predict(X_test)

    # Évaluer les performances du meilleur modèle
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred)

    # Enregistrer les hyperparamètres et les scores dans MLflow
    mlflow.log_params(best_params)
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1", f1)
    mlflow.log_metric("roc_auc", roc_auc)

NameError: ignored

In [21]:
roc_auc

0.5743426196529704

In [22]:
# Calculer la matrice de confusion
confusion_mat = confusion_matrix(y_test, y_pred)

# Afficher la matrice de confusion
print("Matrice de confusion :")
print(confusion_mat)

Matrice de confusion :
[[55545 10740]
 [ 3423  1543]]


In [25]:
X = df.drop('TARGET', axis=1)
y = df['TARGET']

# Divisez les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Appliquer SMOTE uniquement sur l'ensemble d'entraînement
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Définir les hyperparamètres à rechercher
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

from sklearn.metrics import make_scorer

def custom_loss(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    fn_weight = 50  # poids pour les faux négatifs
    fp_weight = 50  # poids pour les faux positifs
    return cm[0, 1] * fp_weight + cm[1, 0] * fn_weight

custom_scorer = make_scorer(custom_loss, greater_is_better=False)

# Créer une instance de DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)

# Effectuer la recherche par grille avec validation croisée
with mlflow.start_run():
    grid_search = GridSearchCV(dt, param_grid, scoring=custom_scorer, cv=2)
    grid_search.fit(X_train_resampled, y_train_resampled)

    # Obtenir les meilleurs hyperparamètres et le meilleur modèle
    best_params = grid_search.best_params_
    best_model = grid_search.best_estimator_

    # Faire des prédictions sur l'ensemble de test avec le meilleur modèle
    y_pred = best_model.predict(X_test)

    # Évaluer les performances du meilleur modèle
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred)

    # Enregistrer les hyperparamètres et les scores dans MLflow
    mlflow.log_params(best_params)
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1", f1)
    mlflow.log_metric("roc_auc", roc_auc)

In [26]:
# Calculer la matrice de confusion
confusion_mat = confusion_matrix(y_test, y_pred)

# Afficher la matrice de confusion
print("Matrice de confusion 20x:")
print(confusion_mat)

Matrice de confusion 20x:
[[55545 10740]
 [ 3423  1543]]


In [31]:
y_test.shape

(71251,)

In [11]:
df1=pd.read_csv("/content/drive/MyDrive/OPENCLASSROOM/PROJET 7/top_95_features.csv")

In [12]:
non_finite_values = np.isfinite(df1)
df1 = df1.replace([np.inf, -np.inf], np.nan).dropna()

In [7]:
X = df1.drop('TARGET', axis=1)
y = df1['TARGET']

# Divisez les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Appliquer SMOTE uniquement sur l'ensemble d'entraînement
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Définir les hyperparamètres à rechercher
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

def custom_loss(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    fn_weight = 10.0  # poids pour les faux négatifs
    fp_weight = 10.0  # poids pour les faux positifs
    return cm[0, 1] * fp_weight + cm[1, 0] * fn_weight

custom_scorer = make_scorer(custom_loss, greater_is_better=False)

# Créer une instance de DecisionTreeClassifier
rfc = RandomForestClassifier(random_state=42)

# Effectuer la recherche par grille avec validation croisée
with mlflow.start_run():
    grid_search = GridSearchCV(rfc, param_grid, scoring=custom_scorer, cv=2, n_jobs=3)
    grid_search.fit(X_train_resampled, y_train_resampled)

    # Obtenir les meilleurs hyperparamètres et le meilleur modèle
    best_params = grid_search.best_params_
    best_model = grid_search.best_estimator_

    # Faire des prédictions sur l'ensemble de test avec le meilleur modèle
    y_pred = best_model.predict(X_test)

    # Évaluer les performances du meilleur modèle
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred)

    # Enregistrer les hyperparamètres et les scores dans MLflow
    mlflow.log_params(best_params)
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1", f1)
    mlflow.log_metric("roc_auc", roc_auc)

In [8]:
confusion_mat = confusion_matrix(y_test, y_pred)
confusion_mat

array([[65889,   392],
       [ 4821,   145]])

In [9]:
roc_auc

0.5116421682072757

In [13]:
X = df1.drop('TARGET', axis=1)
y = df1['TARGET']

# Divisez les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Appliquer SMOTE uniquement sur l'ensemble d'entraînement
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Définir les hyperparamètres à rechercher
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [4],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

def custom_loss(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    fn_weight = 10.0  # poids pour les faux négatifs
    fp_weight = 10.0  # poids pour les faux positifs
    return cm[0, 1] * fp_weight + cm[1, 0] * fn_weight

custom_scorer = make_scorer(custom_loss, greater_is_better=False)

# Créer une instance de DecisionTreeClassifier
rfc = RandomForestClassifier(random_state=42)

# Effectuer la recherche par grille avec validation croisée
with mlflow.start_run():
    grid_search = GridSearchCV(rfc, param_grid, scoring=custom_scorer, cv=2, n_jobs=3)
    grid_search.fit(X_train_resampled, y_train_resampled)

    # Obtenir les meilleurs hyperparamètres et le meilleur modèle
    best_params = grid_search.best_params_
    best_model = grid_search.best_estimator_

    # Faire des prédictions sur l'ensemble de test avec le meilleur modèle
    y_pred = best_model.predict(X_test)

    # Évaluer les performances du meilleur modèle
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred)

    # Enregistrer les hyperparamètres et les scores dans MLflow
    mlflow.log_params(best_params)
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1", f1)
    mlflow.log_metric("roc_auc", roc_auc)

In [14]:
confusion_mat = confusion_matrix(y_test, y_pred)
confusion_mat

array([[60970,  5311],
       [ 3840,  1126]])

In [15]:
roc_auc

0.5733066504590109

In [10]:
X = df1.drop('TARGET', axis=1)
y = df1['TARGET']

# Divisez les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Appliquer SMOTE uniquement sur l'ensemble d'entraînement
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Définir les hyperparamètres à rechercher
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Créer une instance de DecisionTreeClassifier
rfc = RandomForestClassifier(random_state=42)

# Effectuer la recherche par grille avec validation croisée
with mlflow.start_run():
    grid_search = GridSearchCV(rfc, param_grid, scoring="roc_auc", cv=2, n_jobs=3)
    grid_search.fit(X_train_resampled, y_train_resampled)

    # Obtenir les meilleurs hyperparamètres et le meilleur modèle
    best_params = grid_search.best_params_
    best_model = grid_search.best_estimator_

    # Faire des prédictions sur l'ensemble de test avec le meilleur modèle
    y_pred = best_model.predict(X_test)

    # Évaluer les performances du meilleur modèle
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred)

    # Enregistrer les hyperparamètres et les scores dans MLflow
    mlflow.log_params(best_params)
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1", f1)
    mlflow.log_metric("roc_auc", roc_auc)

In [11]:
roc_auc

0.5254595296537146

In [12]:
from xgboost import XGBClassifier

X = df1.drop('TARGET', axis=1)
y = df1['TARGET']

# Divisez les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Appliquer SMOTE uniquement sur l'ensemble d'entraînement
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Définir les hyperparamètres à rechercher
param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [100, 200, 300],
    'gamma': [0, 0.1, 0.2]
}

# Créer une instance de XGBClassifier
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)

# Effectuer la recherche par grille avec validation croisée
with mlflow.start_run(run_name="XGBOOST"):
    grid_search = GridSearchCV(xgb, param_grid, scoring='roc_auc', cv=2, n_jobs=3)
    grid_search.fit(X_train_resampled, y_train_resampled)

    # Obtenir les meilleurs hyperparamètres et le meilleur modèle
    best_params = grid_search.best_params_
    best_model = grid_search.best_estimator_

    # Faire des prédictions sur l'ensemble de test avec le meilleur modèle
    y_pred = best_model.predict(X_test)

    # Évaluer les performances du meilleur modèle
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred)

    # Enregistrer les hyperparamètres et les scores dans MLflow
    mlflow.log_params(best_params)
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1", f1)
    mlflow.log_metric("roc_auc", roc_auc)



/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


make_scorer utoliser ça pour créer mon score à moi . augmenter *10 les erreurs de type 1 et 2 dans la matrice de confusion. Utiliser un dummy classifier une fois mon modele fait, puis le faire tourner sur un algo optimisé.Graphiques: mettre features importance; mettre une jauge, faire des histogrammes. dashboard pour le client, facile à comprendre. un truc non opti pour neophyte, et un truc opti pour dire pourquoi.

decision tree weak modele. Il overfit.
Intéressant sur pbl complexe si on en utilise plein: random forest.

In [3]:
df2=pd.read_csv("/content/drive/MyDrive/OPENCLASSROOM/PROJET 7/fichier_complet.csv")

In [4]:
non_finite_values = np.isfinite(df2)
df2 = df2.replace([np.inf, -np.inf], np.nan).dropna()

In [12]:
X = df2.drop('TARGET', axis=1)
y = df2['TARGET']

# Divisez les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Définir les hyperparamètres à rechercher
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [4],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

def custom_loss(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    fn_weight = 10.0  # poids pour les faux négatifs
    fp_weight = 10.0  # poids pour les faux positifs
    return cm[0, 1] * fp_weight + cm[1, 0] * fn_weight

custom_scorer = make_scorer(custom_loss, greater_is_better=False)

# Créer une instance de DecisionTreeClassifier
rfc = RandomForestClassifier(random_state=42)

# Effectuer la recherche par grille avec validation croisée
with mlflow.start_run():
    grid_search = GridSearchCV(rfc, param_grid, scoring=custom_scorer, cv=2, n_jobs=3)
    grid_search.fit(X_train_resampled, y_train_resampled)

    # Obtenir les meilleurs hyperparamètres et le meilleur modèle
    best_params = grid_search.best_params_
    best_model = grid_search.best_estimator_

    # Faire des prédictions sur l'ensemble de test avec le meilleur modèle
    y_pred = best_model.predict(X_test)

    # Évaluer les performances du meilleur modèle
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred)

    # Enregistrer les hyperparamètres et les scores dans MLflow
    mlflow.log_params(best_params)
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1", f1)
    mlflow.log_metric("roc_auc", roc_auc)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:952: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr

In [13]:
confusion_mat = confusion_matrix(y_test, y_pred)
confusion_mat

array([[8]])